In [1]:
import train
import numpy as np
import api.capital_features_api as cf
import pandas as pd
import mplfinance as mpf
from datetime import datetime
import joblib
import matplotlib.pyplot as plt

In [2]:
train_ud_data = cf.get_train_data(cf.docker_train_data_url, train_config_id=1585649)
validation_ud_data = cf.get_train_data(cf.docker_train_data_url, train_config_id=1585650)

In [3]:
Y_ud_train = train_ud_data['upSignals']
Y_ud_val = validation_ud_data['upSignals']

In [4]:
train_ud_dataset, validation_ud_dataset, scaler_ud = train.to_dataset(train_ud_data, validation_ud_data, Y_ud_train, Y_ud_val)
weights_ud = train.to_weights(Y_ud_train)

In [5]:
model_ud, callbacks_ud = train.get_model(5, 40, 'spy500-200_model_ud_40_ty')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 128)            86528     
                                                                 
 dropout (Dropout)           (None, 5, 128)            0         
                                                                 
 batch_normalization (BatchN  (None, 5, 128)           512       
 ormalization)                                                   
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 batch_normalization_1 (Batc  (None, 64)               256       
 hNormalization)                                        

In [ ]:
his_ud = model_ud.fit(train_ud_dataset, epochs=10000, validation_data=validation_ud_dataset, verbose=1, callbacks=callbacks_ud, class_weight=weights_ud)

Epoch 1/10000
7680/7680 [==============================] - ETA: 0s - loss: 0.7075 - accuracy: 0.5283 - precision: 0.5267 - recall: 0.5567
Epoch 00001: val_loss improved from inf to 0.69322, saving model to ./epoch/spy500-200_model_ud_40_ty/model_ 1.ckpt
7680/7680 [==============================] - 165s 21ms/step - loss: 0.7075 - accuracy: 0.5283 - precision: 0.5267 - recall: 0.5567 - val_loss: 0.6932 - val_accuracy: 0.5281 - val_precision: 0.5288 - val_recall: 0.5163 - lr: 0.0010
Epoch 2/10000
7679/7680 [============================>.] - ETA: 0s - loss: 0.6909 - accuracy: 0.5387 - precision: 0.5356 - recall: 0.5830
Epoch 00002: val_loss improved from 0.69322 to 0.69229, saving model to ./epoch/spy500-200_model_ud_40_ty/model_ 2.ckpt
7680/7680 [==============================] - 160s 21ms/step - loss: 0.6909 - accuracy: 0.5387 - precision: 0.5356 - recall: 0.5830 - val_loss: 0.6923 - val_accuracy: 0.5300 - val_precision: 0.5230 - val_recall: 0.6836 - lr: 0.0010
Epoch 3/10000
7679/7680 [=

In [ ]:
plt.plot(his_ud.history['accuracy'], label='ud_accuracy')
plt.plot(his_ud.history['val_accuracy'], label='ud_val_accuracy')
plt.legend()
plt.show()

In [ ]:
model_ud.save('./model/spy500-200_model_ud_40_ty.h5')
model_ud.save('./model/spy500-200_model_ud_40_ty', save_format='tf')
joblib.dump(scaler_ud, './model/spy500-200_model_up_40_ty_scaler_ud.save')
train.save_scaler('./model/spy500-200_model_up_40_ty_scaler_ud.json', scaler_ud)

In [ ]:
prediction_data = cf.get_prediction_data(cf.docker_prediction_data_url, train_config_id=7, start="2016-07-11", end=datetime.now().strftime("%Y-%m-%d"), code='QQQ')
kline = cf.get_kline_histroy(cf.docker_kline_history_url, code="QQQ", start="2016-07-11", end=datetime.now().strftime("%Y-%m-%d"))
df = cf.his_to_df(kline)

In [ ]:
X_prediction_scaled = train.to_prediction_scaled(prediction_data, scaler_ud)
Y_ud_prediction = model_ud.predict(X_prediction_scaled)

In [ ]:
predictions_ud_series = pd.Series(Y_ud_prediction.flatten(), index=df.index)
buy_signals = np.full(df.shape[0], np.nan)  # 含 NaN 的数组，与 df 的长度一致
buy_signals[predictions_ud_series > 0.9] = df['low'][predictions_ud_series > 0.9] * 0.99
buy_markers = mpf.make_addplot(buy_signals, type='scatter', markersize=50, marker='^', color='g')

sell_signals = np.full(df.shape[0], np.nan)  # 含 NaN 的数组，与 df 的长度一致
sell_signals[predictions_ud_series < 0.1] = df['high'][predictions_ud_series < 0.1] * 1.01
sell_markers = mpf.make_addplot(sell_signals, type='scatter', markersize=50, marker='o', color='r')

mpf.plot(df, type='candle', style='charles', addplot=[buy_markers, sell_markers], volume=True, figsize=(24, 16))